In [ ]:
from tensorflow import keras
from keras.utils import np_utils
import cv2
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import random
np.random.seed(10)

In [ ]:
################################################load data and process################################################

In [ ]:
#padding with 0 to get 1:1 size ratio
def img_padding(img):
    x, y = img.shape
    size_diff = abs(x-y)
    pad_size = int(size_diff/2)
    if(x>y):
        pad = np.zeros((x, pad_size),dtype=int)
        img_1_1 = np.concatenate((pad,img,pad), axis = 1)
        if(size_diff%2 != 0):
            pad1 = np.zeros((x, 1),dtype=int)
            img_1_1 = np.concatenate((img_1_1,pad1), axis = 1)
    elif(y>x):
        pad = np.zeros((pad_size, y),dtype=int)
        img_1_1 = np.concatenate((pad,img,pad), axis = 0)
        if(size_diff%2 != 0):
            pad1 = np.zeros((1, y),dtype=int)
            img_1_1 = np.concatenate((img_1_1,pad1), axis = 0)
    else:
        img_1_1 = img

    return img_1_1

In [ ]:
# this function is for read image,the input is directory name (for RGB)    
def read_directory(directory_name):
    array_of_img = [] # this if for store all of the image data
    dim = (224,224)
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r"./"+directory_name):
        #print(filename) #just for test
        #img is used to store the image data 
        img = cv2.imread(directory_name + "/" + filename, cv2.IMREAD_GRAYSCALE)
        img = img_padding(img)
        img = img.astype(np.uint8)
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        array_of_img.append(img)
    return array_of_img

In [ ]:
#read feature
truck_X = read_directory("training_imgs/RGB_imgs/cap")
truck_X = np.asarray(truck_X)
truck_X = truck_X.reshape((len(truck_X), 224, 224, 1))
forklift_X = read_directory("training_imgs/RGB_imgs/flashlight")
forklift_X = np.asarray(forklift_X)
forklift_X = forklift_X.reshape((len(forklift_X), 224, 224, 1))
negative_X = read_directory("training_imgs/RGB_imgs/soda_can")
negative_X = np.asarray(negative_X)
negative_X = negative_X.reshape((len(negative_X), 224, 224, 1))

print(truck_X.shape)
print(forklift_X.shape) 
print(negative_X.shape)

print(type(negative_X))

(697, 224, 224, 1)
(697, 224, 224, 1)
(700, 224, 224, 1)
<class 'numpy.ndarray'>


In [ ]:
#############################################feature################################################

In [ ]:
#traning and testing feature              #or use the "train_test_split" function from sklearn.model as down below
truck_X_train = truck_X[-100:]
truck_X_test = truck_X[0:100]
forklift_X_train = forklift_X[-100:]
forklift_X_test = forklift_X[0:100]
negative_X_train = negative_X[-100:]
negative_X_test = negative_X[0:100]

#from sklearn.model_selection import train_test_split
#train_images, validation_images, train_labels, validation_labels = train_test_split(train_X, train_Y_onehot, test_size=0.2, random_state=42)

print(truck_X_train.shape, truck_X_test.shape)
print(forklift_X_train.shape, forklift_X_test.shape)
print(negative_X_train.shape, negative_X_test.shape)

(100, 224, 224, 1) (100, 224, 224, 1)
(100, 224, 224, 1) (100, 224, 224, 1)
(100, 224, 224, 1) (100, 224, 224, 1)


In [ ]:
#concatenate 4 categories into training and testing feature
train_X = np.concatenate(( truck_X_train, forklift_X_train, negative_X_train), axis=0)
test_X = np.concatenate(( truck_X_test, forklift_X_test, negative_X_test), axis=0)
print(train_X.shape, test_X.shape)

(300, 224, 224, 1) (300, 224, 224, 1)


In [ ]:
#############################################label################################################

In [ ]:
#training and testing label
truck_Y_train = np.ones(truck_X_train.shape[0])*0
truck_Y_test = np.ones(truck_X_test.shape[0])*0

forklift_Y_train = np.ones(forklift_X_train.shape[0])*1
forklift_Y_test = np.ones(forklift_X_test.shape[0])*1

negative_Y_train = np.ones(negative_X_train.shape[0])*2
negative_Y_test = np.ones(negative_X_test.shape[0])*2


print(truck_Y_train.shape, truck_Y_test.shape)
print(forklift_Y_train.shape, forklift_Y_test.shape)
print(negative_Y_train.shape, negative_Y_test.shape)

(100,) (100,)
(100,) (100,)
(100,) (100,)


In [ ]:
#concatenate 4 categories into training and testing label
train_Y = np.concatenate(( truck_Y_train, forklift_Y_train, negative_Y_train), axis=0)
test_Y = np.concatenate(( truck_Y_test, forklift_Y_test, negative_Y_test), axis=0)
print(train_Y.shape, test_Y.shape)

(300,) (300,)


In [ ]:
#############################################shuffle################################################

In [ ]:
#shuffle the training feature and label
temp = list(zip(train_X, train_Y))
random.shuffle(temp)
train_X, train_Y = zip(*temp)
train_X = np.asarray(train_X)
train_Y = np.asarray(train_Y)
print(train_X.shape, train_Y.shape)

(300, 224, 224, 1) (300,)


In [ ]:
#shuffle the testing feature and label
temp = list(zip(test_X, test_Y))
random.shuffle(temp)
test_X, test_Y = zip(*temp)
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)
print(test_X.shape, test_Y.shape)

(300, 224, 224, 1) (300,)


In [ ]:
#reform training and testing label into one hot format
train_Y_onehot = np_utils.to_categorical(train_Y)
test_Y_onehot = np_utils.to_categorical(test_Y)
print(train_Y_onehot.shape, test_Y_onehot.shape)

(300, 3) (300, 3)


In [ ]:
#############################################model################################################

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten

In [ ]:
model = Sequential()
model.add( Conv2D( input_shape = (224, 224, 1), filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 64, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( MaxPooling2D( pool_size = (2, 2) ) )

model.add( Conv2D( filters = 128, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 128, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( MaxPooling2D( pool_size = (2, 2) ) )

model.add( Conv2D( filters = 256, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 256, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 256, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( MaxPooling2D( pool_size = (2, 2) ) )

model.add( Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( MaxPooling2D( pool_size = (2, 2) ) )

model.add( Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( Conv2D( filters = 512, kernel_size = (3, 3), padding = 'same', activation = 'relu'))
model.add( MaxPooling2D( pool_size = (2, 2) ) )

model.add( Flatten() )
model.add( Dense( units = 4096, kernel_initializer = 'normal', activation = 'relu'))
model.add( Dense( units = 4096, kernel_initializer = 'normal', activation = 'relu'))
model.add( Dense( units = 3, kernel_initializer = 'normal', activation = 'softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(x = train_X, y = train_Y_onehot, validation_split = 0.2, epochs = 10, batch_size = 200, verbose = 2)

Epoch 1/10
2/2 - 18s - loss: 5597.3950 - accuracy: 0.2917 - val_loss: 8.6958 - val_accuracy: 0.2667
Epoch 2/10
2/2 - 20s - loss: 6.4312 - accuracy: 0.3500 - val_loss: 1.1063 - val_accuracy: 0.2667
Epoch 3/10
2/2 - 18s - loss: 1.0867 - accuracy: 0.3875 - val_loss: 1.0051 - val_accuracy: 0.3833
Epoch 4/10
2/2 - 18s - loss: 1.0911 - accuracy: 0.3125 - val_loss: 1.7064 - val_accuracy: 0.3500
Epoch 5/10
2/2 - 17s - loss: 1.8094 - accuracy: 0.3292 - val_loss: 1.0590 - val_accuracy: 0.3833
Epoch 6/10
2/2 - 18s - loss: 1.0798 - accuracy: 0.3208 - val_loss: 1.0462 - val_accuracy: 0.3833
Epoch 7/10
2/2 - 17s - loss: 1.0409 - accuracy: 0.3458 - val_loss: 0.9437 - val_accuracy: 0.5833
Epoch 8/10
2/2 - 16s - loss: 0.8714 - accuracy: 0.6792 - val_loss: 0.8389 - val_accuracy: 0.4333
Epoch 9/10
2/2 - 16s - loss: 0.8118 - accuracy: 0.4833 - val_loss: 3.0397 - val_accuracy: 0.5000
Epoch 10/10
2/2 - 16s - loss: 2.1337 - accuracy: 0.6000 - val_loss: 0.6295 - val_accuracy: 0.6167


In [ ]:
scores = model.evaluate(test_X, test_Y_onehot)
print(scores[1])

10/10 [==============================] - 22s 2s/step - loss: 0.6593 - accuracy: 0.6733
0.6733333468437195


In [ ]:
prediction_class = model.predict_classes( test_X )
print( prediction_class[40:50] )
print( test_Y_onehot[40:50] )
prediction_proba = model.predict_proba( test_X )
print( prediction_proba[40:50] )

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[2 0 0 2 2 2 0 0 0 0]
[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
Instructions for updating:
Please use `model.predict()` instead.
[[0.2811736  0.29978704 0.41903934]
 [0.94305676 0.05240666 0.00453651]
 [0.8460496  0.14002365 0.01392675]
 [0.15523148 0.26158988 0.58317864]
 [0.11847962 0.21124823 0.6702721 ]
 [0.1185726  0.21288885 0.6685386 ]
 [0.7727207  0.18526119 0.04201809]
 [0.7759165  0.20064881 0.02343459]
 [0.74041855 0.24040914 0.01917237]
 [0.60908437 0.36302218 0.02789344]]


In [ ]:
import pandas as pd
pd.crosstab(test_Y, prediction_class, colnames=['predict'], rownames=['real'])

In [ ]:
model.save('CNN_VGG16.h5')

In [ ]:
test_model = keras.models.load_model('CNN_VGG16.h5')